## Code to install MILC and QUDA on Summit
Oct 27, 2022

In [1]:
import os, glob
import subprocess as sp
import numpy as np
import shutil


In [2]:
from ipywidgets import *

## Build scripts

In [3]:
start_dir=os.getcwd()
print(start_dir)

# Create directory to save install scripts
if not os.path.exists(start_dir+'/install_scripts'):
    os.makedirs(start_dir+'/install_scripts/')

/autofs/nccs-svm1_home1/venkitesh/projects/lqcd/multigrid_lattice/Staggered_multigrid_build/build_summit


In [4]:
dict_pars={}
dict_pars['build_dir']='/gpfs/alpine/lgt104/proj-shared/ayyar/builds_summit/install_Dec9_2022'
# dict_pars['run_dir']='/gpfs/alpine/lgt104/proj-shared/ayyar/runs_summit/runs_Dec9_2022'
dict_pars['mgrid']='-DQUDA_MULTIGRID=ON'

## Create upper level directories
if not os.path.exists(dict_pars['build_dir']):
    os.makedirs(dict_pars['build_dir']+'/QUDA/src/')
    # os.makedirs(dict_pars['build_dir']+'/QUDA/src/quda/')
    # os.makedirs(dict_pars['build_dir']+'/MILC')
    os.makedirs(dict_pars['build_dir']+'/install')

In [5]:
### Setup env
env_bash_strg='''#!/bin/bash
#############

module purge
module load DefApps
module unload xalt/1.2.1
#module load cuda/11.3.1
module load cuda/11.0.3
#module load gcc/7.5.0
module load gcc/9.3.0
module load git/2.31.1
module load cmake/3.23.2
module load nsight-systems/2021.3.1.54
module load nsight-compute/2021.2.1
module load gdrcopy/2.2
module list


export LD_LIBRARY_PATH="$BUILD_DIR/QUDA/build/usqcd/lib:${{LD_LIBRARY_PATH}}"
'''.format(**dict_pars)

In [6]:
build_bash_strg1='''#!/bin/bash
date

#############
## Source environment
source ${{BUILD_DIR}}/install_scripts/setup_env_summit.sh

############
## Build QUDA
QUDA_SRC=${{BUILD_DIR}}/QUDA/src/quda
QUDA_BUILD=${{BUILD_DIR}}/QUDA/build

pushd .
if [ ! -d $QUDA_SRC ]
then
  git clone --branch develop https://github.com/lattice/quda $QUDA_SRC # clone QUDA to the desired directory
else
  cd $QUDA_SRC; git pull; cd ..
fi

if [ ! -d $QUDA_BUILD ]
then
  mkdir $QUDA_BUILD
else
  cd $QUDA_BUILD
  rm -rf *
fi

cd $QUDA_BUILD
cmake -DCMAKE_BUILD_TYPE=RELEASE -DQUDA_DIRAC_DEFAULT_OFF=ON -DQUDA_DIRAC_STAGGERED=ON -DQUDA_GPU_ARCH=sm_70 -DQUDA_DOWNLOAD_USQCD=ON -DQUDA_QIO=ON -DQUDA_QMP=ON -DQUDA_MULTIGRID=ON $QUDA_SRC
nice make -j 4
make install
popd

date
'''.format(**dict_pars)


In [7]:
build_bash_strg2='''#!/bin/bash
#############

## Source environment
script_loc=$BUILD_DIR/install_scripts
source $script_loc/setup_env_summit.sh

############
## Build QUDA
git clone --branch develop https://github.com/milc-qcd/milc_qcd $BUILD_DIR/milc_qcd
'''.format(**dict_pars)

In [8]:
build_bash_strg3='''#!/bin/bash
date

#############
## Source environment
script_loc=$BUILD_DIR/install_scripts
source $script_loc/setup_env_summit.sh

############
## Build QUDA
QUDA_SRC=${{BUILD_DIR}}/QUDA/src/quda
QUDA_BUILD=${{BUILD_DIR}}/QUDA/build

## Build MILC

MILCDIR=${{BUILD_DIR}}/milc_qcd

cd ${{MILCDIR}}/ks_spectrum && cp ${{MILCDIR}}/Makefile ${{MILCDIR}}/ks_spectrum
export PATH_TO_CUDA=/sw/summit/cuda/11.0.3 # NEED TO MAKE THIS MORE ROBUST TO DIFFERENT CUDA VERSIONS, ASSUMES 11.0.3
export USQCD_BUILD=${{QUDA_BUILD}}/usqcd

PATH_TO_CUDA=$PATH_TO_CUDA PATH_TO_QUDA=$QUDA_BUILD PATH_TO_QMP=$USQCD_BUILD PATH_TO_QIO=$USQCD_BUILD POWER9=1 MULTIGRID=1 $MILCDIR/ks_spectrum/./compile_ks_spectrum_hisq_quda.sh

cd $BUILD_DIR
date
'''.format(**dict_pars)

In [9]:
loc1=start_dir+'/install_scripts/'
loc2=dict_pars['build_dir']+'/'
loc1,loc2

('/autofs/nccs-svm1_home1/venkitesh/projects/lqcd/multigrid_lattice/Staggered_multigrid_build/build_summit/install_scripts/',
 '/gpfs/alpine/lgt104/proj-shared/ayyar/builds_summit/install_Dec9_2022/')

In [10]:
fname1='setup_env_summit.sh'
with open (loc1+fname1,'w') as f: f.write(env_bash_strg)

fname2='build_quda.sh'
with open (loc1+fname2,'w') as f: f.write(build_bash_strg1)

fname3='build_milc1.sh'
with open (loc1+fname3,'w') as f: f.write(build_bash_strg2)

fname4='build_milc2.sh'
with open (loc1+fname4,'w') as f: f.write(build_bash_strg3)

print(fname1,fname2,fname3,fname4)

setup_env_summit.sh build_quda.sh build_milc1.sh build_milc2.sh


In [11]:
%%bash -s "$fname1" "$fname2" "$fname3" "$fname4" "$loc1" "$loc2" ## Use python variables in bash

# ls -al $5$1
# ls -al $5$4

chmod +x $5/$1
chmod +x $5/$2
chmod +x $5/$3
chmod +x $5/$4

cp -r $5 $6

### Run the build
./install_scripts/build_quda.sh 2>&1 | tee op_quda.out \
./install_scripts/build_milc1.sh 2>&1 | tee op_milc1.out \
< make some changes to files> \
./install_scripts/build_milc2.sh 2>&1 | tee op_milc2.out

In [12]:
# cat install_scripts/build_milc.sh